In [1]:
from models import *

In [2]:
one_hot = True
write = False
board = Board(width = 31, height = 31, spawn_rate=1, max_spawn_batch=5, one_hot = one_hot, write=write)
if write:
    #Clean data file
    with open("data.txt", "w") as file:
        file.write("")
for i in range(50):
    #print('Step', i)
    board.step()

In [3]:
%%capture
paleta = np.array(
    [
    # 0 -> highway
    [30 , 30 , 30 ],  
    # 1 -> up traffic light, red
    [255, 0  , 0  ],
    # 2 -> down traffic light, green
    [0  , 255, 0  ],
    # 3 -> road, dark green
    [0  , 60, 0  ],
    # 4 -> car looking up, purple
    [255, 0  , 255],
    # 5 -> car looking down, orange
    [255, 165, 0  ],
    # 6 -> car looking left, pink
    [255, 192, 203],
    # 7 -> car looking right, cyan
    [0  , 255, 255],
    # 8 -> lane
    [255, 255, 255]
    ])
all_grid = board.datacollector.get_model_vars_dataframe()
pinta_me = lambda matriz: paleta[matriz.astype(int)]
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(pinta_me(all_grid.iloc[0][0]))
def animate(i):
    patch.set_data(pinta_me(all_grid.iloc[i][0]))
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

TypeError: Invalid shape (31, 31, 11, 3) for image data

In [4]:
all = board.datacollector.get_model_vars_dataframe()
# Save the 3d one hot encoded grid to txt for visualization
with open("grid.txt", "w") as file:
    for i in range(len(all)):
        for j in range(len(all.iloc[i][0])):
            for k in range(len(all.iloc[i][0][j])):
                file.write(str(all.iloc[i][0][j][k]) + " ")
            file.write("\n")
all


,Grid
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
6,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
7,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
8,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
9,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [ ]:
# Success rate
print("Success rate:", "{:.2f}".format(board.successful_trips / (board.successful_trips + board.crashes)*100)+"%")
print(f"Crashes: {int(board.crashes)}")
print(f"Successful trips: {board.successful_trips}")
print(f"Time stuck: {board.time_stuck}")

Success rate: 87.74%
Crashes: 109
Successful trips: 780
Time stuck: 13404


In [ ]:
anim

# M3. Interacciones entre agentes (Etapa Reto: 1.1)

Narhari Olalde Guajardo			| A01284077<br>
Guillermo Tafoya Milo			| A01633790<br>
Mauricio Maqueo Huerta			| A01620649<br>
Emiliano Yoltic Martinez 		| A01620000<br>

## Responder a las preguntas de la etapa 1.1 del reto:

### ¿Cómo se modela la circulación de un automóvil en un ambiente urbano?

Para el desarrollo de este proyecto el comportamiento de los vehículos estará determinado mediante inteligencia artificial e interactuando en el entorno mediante sensores ubicados en el juego de Unity en forma de semáforos.

### ¿Cómo se modela la circulación de un grupo de automóviles en un ambiente urbano?

La idea principal del proyecto es modelar el comportamiento del tráfico en un entorno controlado en el cual podamos modificar los parámetros por los cuales los semáforos se rigen al igual que poder controlar la velocidad y población de los coches. 


1. Mientras no haya un vehículo cercano, el semáforo estará en luz amarilla. 
2. Cuando un vehículo se acerque a la intersección, enviará un mensaje con el tiempo estimado de arribo.
3. El semáforo dará luz verde al semáforo más cercano y establecerá un programa de luces a partir de ese punto para el resto de los vehículos.

#### Agentes:
1. Coches
2. Semáforos

#### Diagrama de estados
1. Mientras no haya un vehículo cercano, el semáforo estará en luz amarilla. 
No vehiculo = semaforo en amarillo
2. Cuando un vehículo se acerque a la intersección, enviará un mensaje con el tiempo estimado de arribo.
Vehículo cerca = se manda a tiempo de llegada estimado a semáforo más cercano 
3. El semáforo dará luz verde al semáforo más cercano y establecerá un programa de luces a partir de ese punto para el resto de los vehículos.
